In [1]:
import requests
import pandas as pd
import json

In [3]:
from gcapi import GCapiClient

In [4]:
api = GCapiClient(username='DA438287', password='trade123', appkey='J.Park', proxies=None)

In [3]:
api.get_account_info(get=None)

{'LogonUserName': 'Mr Joonsung Park  - Test',
 'ClientAccountId': 402201187,
 'CultureId': 69,
 'ClientAccountCurrency': 'EUR',
 'AccountOperatorId': 401375842,
 'TradingAccounts': [{'TradingAccountId': 402710096,
   'TradingAccountCode': 'DA438287',
   'TradingAccountStatus': 'Open',
   'TradingAccountType': 'CFD'}],
 'PersonalEmailAddress': 'bear.park1220@gmail.com',
 'HasMultipleEmailAddresses': False,
 'AccountHolders': [{'LegalPartyId': 404427574,
   'Name': 'Mr Joonsung Park  - Test'}],
 'ClientTypeId': 1,
 'LinkedClientAccounts': [],
 'IsNfaEnabledClient': False,
 'IsFifo': None,
 'DaysUntilExpiryForDemo': 32.163058480191,
 'LegalPartyUniqueReference': 8110413,
 'Is2FAEnabledAO': False,
 'Regulatory': {'IsMiFIDRegulator': True,
  'IsPiisProvided': False,
  'ClientAccountCreationDate': '/Date(1588372088473)/'},
 'IsDMAClient': False}

In [4]:
api.get_margin_info(get=None)

{'Cash': 11412.56,
 'Margin': 0.0,
 'MarginIndicator': -1.0,
 'NetEquity': 11412.56,
 'OpenTradeEquity': 0.0,
 'TradableFunds': 11412.56,
 'PendingFunds': 0.0,
 'TradingResource': 11412.56,
 'TotalMarginRequirement': 0.0,
 'CurrencyId': 4,
 'CurrencyIsoCode': 'EUR'}

In [14]:
api.get_ohlc(market_id=401166447, num_ticks=100, interval="HOUR", span=1, from_ts=None, to_ts=None)

BarDate     Open     High      Low    Close
0  2020-04-28 16:00:00  1.08437  1.08580  1.08303  1.08325
1  2020-04-28 17:00:00  1.08331  1.08465  1.08251  1.08365
2  2020-04-28 18:00:00  1.08363  1.08386  1.08254  1.08317
3  2020-04-28 19:00:00  1.08318  1.08372  1.08270  1.08358
4  2020-04-28 20:00:00  1.08357  1.08400  1.08331  1.08359
..                 ...      ...      ...      ...      ...
95 2020-05-04 15:00:00  1.09349  1.09381  1.09218  1.09267
96 2020-05-04 16:00:00  1.09264  1.09355  1.09038  1.09164
97 2020-05-04 17:00:00  1.09163  1.09179  1.09006  1.09105
98 2020-05-04 18:00:00  1.09107  1.09111  1.08958  1.09002
99 2020-05-04 19:00:00  1.09004  1.09064  1.08970  1.09022

[100 rows x 5 columns]

In [11]:
rest_url = 'https://ciapi.cityindex.com/TradingAPI'
headers = {'Content-Type':'application/json'}
data = {
    "UserName": 'DA438287',
    "Password": 'trade123',
    'AppKey':'J.Park'
}
r=requests.post(rest_url + '/session', headers=headers, proxies=None, json=data)


In [12]:
resp = r.json()

In [13]:
resp

{'Session': 'a64eb224-a768-4284-a5b7-0f83cf403f7a',
 'PasswordChangeRequired': False,
 'AllowedAccountOperator': False,
 'StatusCode': 1,
 'AdditionalInfo': None,
 'Is2FAEnabled': False,
 'TwoFAToken': None,
 'Additional2FAMethods': None,
 'UserType': 1}

In [14]:
resp['StatusCode']

1

In [15]:
with requests.Session() as s:
    s.headers = headers
session=s

#### Clear
It is understandable until this point.    
To use the REST API, I need to assign the values of 'URL','HEADERS' and 'DATA' dictionaries.   
It is entering in the system using request.post step.   

#### Problem
Using the library, I can get the market info. It is searching the MARKET ID.   
When I type the code by myself, it is not working correctly.

#### Things to DO
https://github.com/rickykim93/gcapi-python/blob/master/gcapi/gcapi_client.py 
Study from the def get_prices

In [16]:
price_type='MID'
market_id = 401449250
num_ticks=1000
interval='HOUR'
span=1
#from_ts = '1234'
now_ts = datetime.now()
print(now_ts)
r = session.get(rest_url + f'/market/{market_id}/barhistory?interval={interval}&span={span}&PriceBars={num_ticks}')
r.json()
#session.get(rest_url + '/market/401449250/tickhistoryafter?interval=HOUR&span=1&maxResults=10&toTimeStampUTC=2020-05-05 15:14:28.197478')

2020-05-05 15:43:21.587004


{'HttpStatus': 401, 'ErrorMessage': 'Session is not valid', 'ErrorCode': 4011}

In [23]:
def get_ohlc(market_id=None, num_ticks=None, interval="HOUR", span=1, from_ts=None, to_ts=None):
		"""
		Get the open, high, low, close of a specific market_id
		:param market_id: market ID
		:param num_ticks: number of price ticks/data to retrieve
		:param interval: MINUTE, HOUR or DAY tick interval
		:param span: it can be a combination of span with interval, 1Hour, 15 MINUTE
		:param from_ts: from timestamp UTC
		:param to_ts: to timestamp UTC
		:return: ohlc dataframe
		"""


		if market_id is None:
			market_id = market_id
		interval = check_interval(interval)
		span = check_span(interval, span)
		if from_ts is not None and to_ts is not None:
			r = session.get(
				rest_url + f'/market/{market_id}/barhistorybetween?interval={interval}&span={span}fromTimeStampUTC={from_ts}&toTimestampUTC={to_ts}')
		else:
			if not num_ticks:
				num_ticks=1
			if from_ts is not None:
				r = session.get(
					rest_url + f'/market/{market_id}/barhistorybefore?interval={interval}&span={span}&maxResults={num_ticks}&toTimeStampUTC={to_ts}')
			elif to_ts is not None:
				r = session.get(
					rest_url + f'/market/{market_id}/tickhistoryafter?interval={interval}&span={span}&maxResults={num_ticks}&fromTimeStampUTC={from_ts}')
			else:
				r = session.get(rest_url + f'/market/{market_id}/barhistory?interval={interval}&span={span}&PriceBars={num_ticks}')
		resp = json.loads(r.text)

		try:
			if num_ticks==1:
				return resp.get('PriceBars')[0]
			else:
				df_ohlc = pd.DataFrame(resp.get('PriceBars'))
				df_ohlc['BarDate'] = df_ohlc['BarDate'].map(format_date)
				return df_ohlc
		except:
			raise GCapiException(resp)


In [24]:
get_ohlc( market_id= 401449250, num_ticks=100, interval='HOUR', span=1)

NameError: name 'check_interval' is not defined

In [2]:
session.get('https://ciapi.cityindex.com/TradingAPI/market/search?SearchByMarketName=TRUE&Query=GBP%2FUSD&MaxResults=10')

<Response [401]>

In [13]:
requests.get('https://ciapi.cityindex.com/TradingAPI/market/information')

<Response [404]>

In [20]:
requests.get('https://ciapi.cityindex.com/TradingAPI/market/search?SearchByMarketName=TRUE&Query=GBP%2FUSD&MaxResults=2&includeOptions=False')

<Response [401]>

#### What I understand and can use

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
#from gcapi import GCapiClient
#api = GCapiClient(username='DA438287', password='trade123', appkey='J.Park', proxies=None)

rest_url = 'https://ciapi.cityindex.com/TradingAPI'
headers = {'Content-Type':'application/json'}
data = {
    "UserName": 'DA438287',
    "Password": 'trade123',
    'AppKey':'J.Park'
}
r=requests.post(rest_url + '/session', headers=headers, proxies=None, json=data)

resp = r.json()
print('If Okay, it is 1 =', resp['StatusCode'])

with requests.Session() as s:
    s.headers = headers
session=s

api.get_market_info('EUR/USD',get=None)


If Okay, it is 1 = 1


NameError: name 'api' is not defined